In [23]:
import tkinter as tk
import random
import numpy as np
from tkinter import ttk

Problem statement:

    The goal of the code is to simulate the movement of people in a grid environment. The grid contains obstacles, rewards, and free spaces. People move around randomly in the grid, collecting rewards, while avoiding obstacles.

Mathematical formulation:

    Grid: The grid is a 2D matrix of size n x m.
    Obstacles: Randomly placed in the grid, marked by #.
    Rewards: Randomly placed, marked by R.
    People: Randomly placed in free spaces, marked by P.
    Movement: People move one step at a time in random directions (right, down, left, up) until they find an empty space or collect a reward.

Data requirements:

    n, m: Dimensions of the grid.
    num_people: Number of people to be placed in the grid.
    num_obstacles: Number of obstacles to be placed in the grid.
    num_rewards: Number of rewards to be placed in the grid.
    steps: Number of steps for the simulation.

Success metrics:

    Total number of rewards collected by all people after the simulation ends.
    Grid configuration and the movement of people.

In [3]:

def create_grid(n, m, num_obstacles, num_rewards):
    grid = [[" " for _ in range(m)] for _ in range(n)]
    
    # Place obstacles
    obstacles = set()
    while len(obstacles) < num_obstacles:
        x, y = random.randint(0, n-1), random.randint(0, m-1)
        if (x, y) not in obstacles:
            grid[x][y] = "#"  # Obstacle
            obstacles.add((x, y))
    
    # Place reward squares
    rewards = set()
    while len(rewards) < num_rewards:
        x, y = random.randint(0, n-1), random.randint(0, m-1)
        if (x, y) not in obstacles and (x, y) not in rewards:
            grid[x][y] = "R"  # Reward
            rewards.add((x, y))
    
    return grid, obstacles, rewards

def place_people(grid, num_people):
    n, m = len(grid), len(grid[0])
    people = {}
    occupied = set()
    
    for i in range(num_people):
        while True:
            x, y = random.randint(0, n-1), random.randint(0, m-1)
            if grid[x][y] == " " and (x, y) not in occupied:
                people[i] = (x, y)
                occupied.add((x, y))
                break
    
    return people

def move_people(grid, people, obstacles, rewards, reward_counter):
    directions = [(0,1), (1,0), (0,-1), (-1,0)]  # Right, Down, Left, Up
    occupied = set(people.values())
    
    for person in people:
        x, y = people[person]
        random.shuffle(directions)
        
        for dx, dy in directions:
            new_x, new_y = x + dx, y + dy
            if 0 <= new_x < len(grid) and 0 <= new_y < len(grid[0]):
                if (new_x, new_y) not in obstacles and (new_x, new_y) not in occupied:
                    people[person] = (new_x, new_y)
                    occupied.remove((x, y))
                    occupied.add((new_x, new_y))
                    
                    if (new_x, new_y) in rewards:
                        reward_counter[0] += 1
                    break  # Move successful
    
    return people

def print_grid(grid, people):
    display_grid = [row[:] for row in grid]
    for person in people.values():
        x, y = person
        display_grid[x][y] = "P"
    
    for row in display_grid:
        print("_".join(row))
    print()

def simulate(n, m, num_people, num_obstacles, num_rewards, steps):
    grid, obstacles, rewards = create_grid(n, m, num_obstacles, num_rewards)
    people = place_people(grid, num_people)
    reward_counter = [0]
    
    for step in range(steps):
        print(f"Step {step+1}:")
        people = move_people(grid, people, obstacles, rewards, reward_counter)
        print_grid(grid, people)
    
    print(f"Total rewards collected: {reward_counter[0]}")




Step 1:
 _ _ _#_ _R_R_#
 _#_ _ _P_ _#_ 
 _R_ _P_ _ _ _ 
 _ _ _ _#_ _ _#
 _ _ _P_#_ _ _R
 _ _ _ _#_ _ _ 
 _ _ _#_ _ _P_#
 _ _R_ _P_ _ _ 

Total rewards collected: 0


Running simulation on random obstacles

In [ ]:
# Example usage
simulate(n=8, m=8, num_people=5, num_obstacles=10, num_rewards=5, steps=1)

Let's allow the user to create their own obstacles to simulate more of a stadium environment.

In [5]:

def create_grid(n, m, obstacles, num_rewards, max_reward):
    grid = [["_" for _ in range(m)] for _ in range(n)]
    
    # Place obstacles
    obstacle_set = set()
    for x, y in obstacles:
        grid[x][y] = "#"  # Obstacle
    
    # Place reward squares with varying rewards
    rewards = {}
    while len(rewards) < num_rewards:
        x, y = random.randint(0, n-1), random.randint(0, m-1)
        if (x, y) not in obstacles and (x, y) not in rewards:
            rewards[(x, y)] = random.randint(1, max_reward)  # Assign a random reward value
            grid[x][y] = str(rewards[(x, y)])  # Display reward value on grid
    
    return grid, obstacle_set, rewards

def place_people(grid, num_people):
    n, m = len(grid), len(grid[0])
    people = {}
    occupied = set()
    
    for i in range(num_people):
        while True:
            x, y = random.randint(0, n-1), random.randint(0, m-1)
            if grid[x][y] == "_" and (x, y) not in occupied:
                people[i] = (x, y)
                occupied.add((x, y))
                break
    
    return people

def move_people(grid, people, obstacles, rewards, reward_counter):
    directions = [(0,1), (1,0), (0,-1), (-1,0)]  # Right, Down, Left, Up
    occupied = set(people.values())
    
    for person in people:
        x, y = people[person]
        random.shuffle(directions)
        
        for dx, dy in directions:
            new_x, new_y = x + dx, y + dy
            if 0 <= new_x < len(grid) and 0 <= new_y < len(grid[0]):
                if (new_x, new_y) not in obstacles and (new_x, new_y) not in occupied:
                    people[person] = (new_x, new_y)
                    occupied.remove((x, y))
                    occupied.add((new_x, new_y))
                    
                    if (new_x, new_y) in rewards:
                        reward_counter[0] += rewards[(new_x, new_y)]
                    break  # Move successful
    
    return people

def distribute_rewards(grid, rewards):
    directions = [(0,1), (1,0), (0,-1), (-1,0)]  # Right, Down, Left, Up
    reward_weights = {k: v for k, v in rewards.items()}  # Copy rewards
    
    queue = list(rewards.keys())
    visited = set(queue)
    while queue:
        x, y = queue.pop(0)
        weight = grid[x][y]
        
        for dx, dy in directions:
            nx, ny = x + dx, y + dy
            if 0 <= nx < len(grid) and 0 <= ny < len(grid[0]) and (nx, ny) not in visited and grid[nx][ny] == "_":
                grid[nx][ny] = str(float(weight) / 4)  # Distribute weight
                queue.append((nx, ny))
                visited.add((nx, ny))
    
    return reward_weights

def print_grid(grid, people):
    display_grid = [row[:] for row in grid]
    for person in people.values():
        x, y = person
        display_grid[x][y] = "P"
    
    for row in display_grid:
        print("_".join(row))
    print()

def simulate(n, m, num_people, num_obstacles, num_rewards, max_reward, steps):
    grid, obstacles, rewards = create_grid(n, m, num_obstacles, num_rewards, max_reward)
    people = place_people(grid, num_people)
    reward_counter = [0]
    
    distribute_rewards(grid, rewards)
    
    for step in range(steps):
        print(f"Step {step+1}:")
        people = move_people(grid, people, obstacles, rewards, reward_counter)
        print_grid(grid, people)
    
    print(f"Total rewards collected: {reward_counter[0]}")

obstacles = {(1, 1), (2, 3), (4, 4)}
# Example usage
simulate(n=10, m=10, num_people=5, num_obstacles=obstacles, num_rewards=5, max_reward=5, steps=10)



Step 1:
1.1920928955078125e-06_4.76837158203125e-06_1.9073486328125e-05_1.1444091796875e-05_4.57763671875e-05_0.00018310546875_0.000732421875_0.0029296875_0.01171875_0.0029296875
4.76837158203125e-06_#_7.62939453125e-05_4.57763671875e-05_0.00018310546875_0.000732421875_0.0029296875_0.01171875_0.046875_0.01171875
1.9073486328125e-05_7.62939453125e-05_0.00030517578125_#_0.000732421875_0.0029296875_0.01171875_0.046875_0.1875_0.046875
7.62939453125e-05_P_0.001220703125_P_0.0029296875_0.01171875_0.046875_0.1875_0.75_0.1875
0.00030517578125_0.001220703125_0.0048828125_0.01953125_#_0.046875_P_0.75_3_0.75
0.001220703125_0.0048828125_0.01953125_0.078125_0.01953125_0.01171875_0.046875_0.1875_0.75_0.1875
0.0048828125_0.01953125_0.078125_0.3125_0.078125_0.01953125_0.015625_0.0625_0.25_0.0625
P_0.078125_0.3125_1.25_0.3125_0.078125_0.0625_0.25_1.0_0.25
0.078125_0.3125_1.25_5_1.25_0.3125_0.25_1.0_4_1.0
0.015625_0.0625_0.25_1_0.25_0.0625_0.125_P_2_0.5

Step 2:
1.1920928955078125e-06_4.76837158203125e-

Let's visualize in a TKinter GUI!

In [19]:
def update_gui(canvas, grid, people, rewards):
    canvas.delete("all")
    cell_size = 40
    for i, row in enumerate(grid):
        for j, cell in enumerate(row):
            color = "white"
            if cell == "#":
                color = "black"
            elif (i, j) in rewards:
                color = "yellow"
            elif not (isinstance(people, int)) and (i, j) in people.values():
                color = "blue"
            canvas.create_rectangle(j * cell_size, i * cell_size, (j + 1) * cell_size, (i + 1) * cell_size, fill=color, outline="gray")
            if cell != "#" and cell != "_":
                canvas.create_text(j * cell_size + 20, i * cell_size + 20, text=str(round(float(cell), 3)), fill="black", font=("Arial", 10, "bold"))
                
def simulate_gui(n, m, num_people, num_obstacles, num_rewards, max_reward, steps):
    grid, obstacles, rewards = create_grid(n, m, num_obstacles, num_rewards, max_reward)
    people = place_people(grid, num_people)
    reward_counter = [0]
    
    distribute_rewards(grid, rewards)
    
    root = tk.Tk()
    root.title("Grid Traversal Simulation")
    best_reward = 0
    best_grid = None
    canvas = tk.Canvas(root, width=m*40, height=n*40)
    canvas.pack()
    step_count = [0]
    def update():
        nonlocal best_reward, best_grid
        if step_count[0] < steps:
            nonlocal people
            people = move_people(grid, people, obstacles, rewards, reward_counter)
            if reward_counter[0] > best_reward:
                best_reward = reward_counter[0]
                best_grid = [row[:] for row in grid]
            update_gui(canvas, grid, people, rewards)
            step_count[0] += 1
            root.after(1, update)
        else:
            print("Final Reward Counter:", reward_counter[0])
            print("Final Grid Configuration:")
            for row in grid:
                print(" ".join(row))
            display_best_grid(best_grid)
            root.destroy()
    
    update_gui(canvas, grid, people, rewards)
    root.after(1000, update)
    root.mainloop()
    
    return reward_counter[0], grid

def display_best_grid(best_grid):
    # Create a new window to display the best grid
    best_window = tk.Toplevel()
    best_window.title("Best Grid - Highest Reward")
    
    best_canvas = tk.Canvas(best_window, width=len(best_grid[0])*40, height=len(best_grid)*40)
    best_canvas.pack()
    
    # Use update_gui to render the best grid
    update_gui(best_canvas, best_grid, {}, {})
# Run the GUI simulation

obstacles = set()

for i in range(8, 13):
    for j in range(13, 18):
        obstacles.add((i, j))
# sim_reward, sim_grid = simulate_gui(n=20, m=30, num_people=20, num_obstacles=obstacles, num_rewards=20, max_reward=5, steps=1000)


# print(sim_reward)
# print(sim_grid)

Now that we have a visualization, let's run 10 simulations and pick out which one worked the best.

In [29]:
max_sim_reward = 0
max_sim_grid = []
max_sim_reward_spots = []
n = 20
m = 30

# Optimization Algorithm: Maximize Revenue on 10 random simulations
# Params include Grid Size, agents, obstacles, rewards, and max reward. Steps are of size 100

for _ in range(10):
    sim_reward, sim_grid = simulate_gui(n=n, m=m, num_people=20, num_obstacles=obstacles, num_rewards=20, max_reward=5, steps=100)
    if sim_reward > max_sim_reward:
        max_sim_reward = sim_reward # Objective Function is total revenue, we are trying to maximize
        max_sim_grid = sim_grid
        max_sim_reward_spots = sim_reward



KeyboardInterrupt: 

max_sim_grid is our grid configuration with the highest reward out of the 10 simulations - we obtain the coordinates for the reward stand's positions

In [22]:
#max_sim_grid

[['3.814697265625e-06',
  '1.52587890625e-05',
  '6.103515625e-05',
  '0.000244140625',
  '0.0009765625',
  '0.00390625',
  '0.015625',
  '0.0625',
  '0.25',
  '1',
  '0.25',
  '0.0625',
  '0.015625',
  '0.00390625',
  '0.0009765625',
  '0.000244140625',
  '0.00018310546875',
  '0.000732421875',
  '0.0029296875',
  '0.01171875',
  '0.046875',
  '0.01171875',
  '0.0029296875',
  '0.0078125',
  '0.03125',
  '0.125',
  '0.25',
  '0.0625',
  '0.015625',
  '0.00390625'],
 ['1.9073486328125e-05',
  '7.62939453125e-05',
  '0.00030517578125',
  '0.001220703125',
  '0.0048828125',
  '0.01953125',
  '0.078125',
  '0.3125',
  '0.0625',
  '0.25',
  '0.0625',
  '0.015625',
  '0.00390625',
  '0.0009765625',
  '0.000244140625',
  '0.00018310546875',
  '0.000732421875',
  '0.0029296875',
  '0.01171875',
  '0.046875',
  '0.1875',
  '0.046875',
  '0.01171875',
  '0.03125',
  '0.125',
  '0.5',
  '1.0',
  '0.25',
  '0.0625',
  '0.015625'],
 ['7.62939453125e-05',
  '0.00030517578125',
  '0.001220703125',
 

In [28]:
np_grid = np.array(max_sim_grid)
np_grid[np_grid == "#"] = 0
np.where(np_grid.astype(float) > 1)

(array([ 2,  2,  2,  3,  3,  3,  3,  4,  7,  8,  9, 10, 10, 11, 11, 11, 11,
        12, 12, 13, 14, 15, 15, 15, 15, 15, 16, 19, 19, 19], dtype=int64),
 array([ 7, 25, 26,  6,  7,  8, 20,  7,  9,  6, 20, 19, 23,  2, 18, 19, 20,
        12, 19,  6,  8,  7,  8,  9, 11, 16,  8,  3,  8, 11], dtype=int64))